تمرین1.6.2 


In [1]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EmpiricalCovariance
from scipy.spatial import distance
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.mixture import BayesianGaussianMixture

In [2]:
I=np.identity(2, dtype = float)

def mixt_model(m,S,P,N):
    l,c=m.shape
    P_acc=[P[0]]
    for i in range(1,c):
        t=P_acc[i-1]+P[i]
        P_acc.append(t)
    
    X=[]
    y=[]
    P_acc=np.array(P_acc)

    for i in range(N):
        t=np.random.rand()
        ind=np.sum(t>P_acc)
        X.append(np.random.multivariate_normal(m[:,ind],S[:,:,ind]))
        y.append(ind+1)
    return np.array(X).T,np.array(y)

def modelGenrealMix(m_init,s_init,weight):
    d=[]
    for i in range(len(m_init)):
        d.append(MultivariateGaussianDistribution(m_init[i],s_init[:,:,i]))
    model = GeneralMixtureModel(d, weights=weight)
    return model

def gauss(x,m,s):
    J,l=m.shape
    p=x.shape
    z=[]
    for j in range(J):
        t=(x-m[j,:]).dot(x-m[j,:]).T
        c=1/(2*np.pi*s[j])**(1/2)
        z.append(c*np.exp(-t/2*s[j]))
    return np.array(z)

def em_alg_function(x,m,s,Pa,e_min):
    x=np.array(x)
    m=np.array(m).T
    p,n=x.shape
    J,n=m.shape
    e=e_min+1
    Q_tot=[]
    e_tot=[]
    iter=0
    
    while (e>e_min):
        iter=iter+1
        P_old=Pa
        m_old=m
        s_old=s
        P=np.zeros([J,p])
        for k in range(p):
            temp=gauss(x[k,:],m,s)
            P_tot=temp.dot(Pa.T)
            for j in range(J):
                P[j,k]=temp[j]*Pa[j]/P_tot
        Q=0
        for k in range(p):
            for j in range(J):
                Q=Q+P[j,k]*(-(n/2)*np.log(2*np.pi*s[j])-np.sum((x[k,:]-m[j,:])**2)/(2*s[j])+np.log(Pa[j]))
        Q_tot.append(Q)

        for j in range(J):
            a=np.zeros([1,n])
            for k in range(p):
                a=a+P[j,k]*x[k,:]
            m[j,:]=a/np.sum(P[j,:])
        
        for j in range(J):
            b=0
            for k in range(p):
                b=b+P[j,k]*((x[k,:]-m[j,:]).dot((x[k,:]-m[j,:]).T))
            
            s[j]=b/(n*np.sum(P[j,:]))
            if s[j]<10**(-10):
                s[j]=0.001
        
        #Determine the a priori probabilities
        for j in range(J):
            a=0
            for k in range(p):
                a=a+P[j,k]
            Pa[j]=a/p
        
        e=np.sum(np.abs(Pa-P_old))+np.sum(np.sum(np.abs(m-m_old)))+np.sum(np.abs(s-s_old))
        e_tot.append(e)

    return [m,s,Pa,iter,Q_tot,e_tot]

def EM_pdf_est(X,y,m_ini,s_ini,P_ini):
    l,N=X.shape
    e_min=10**(-5)
    cl=int(np.max(y))
    acc_tot=[]
    Xs=[]
    for j in range(cl):
        temp=[]
        t=0
        for i in range(N):
            if(y[i]==j+1):
                temp.append(X[:,i])
                t=t+1
        acc_tot.append(t)
        Xs.append(temp)
    P_cl=np.array(acc_tot)/N
    m=[]
    s=[]
    P=[]
    for j in range(cl):
        [mj,sj,Pj,iter1,Q_tot,e_tot]=em_alg_function(Xs[j],m_ini[j],s_ini[j],P_ini[j],e_min)
        m.append(mj)
        s.append(sj)
        P.append(Pj)

    return [m,s,P,P_cl]

def mixt_value(m,S,P,X):
    l=X.shape
    N=1
    m=m.T
    l,c=m.shape
    y=[]
    for i in range(N):
        temp=[]
        for j in range(c):
            t=multivariate_normal.pdf(X[:], mean=m[:,j], cov=S[:,:,j])
            temp.append(t)
        y_temp=np.sum(P*np.array(temp))
        y.append(y_temp+1)
    return y[0]

def mixture_Bayes(m,S,P,P_cl,X):
    cl=len(m)
    l,N=X.shape
    y=[]
    for i in range(N):
        temp=[]
        for j in range(cl):
            t=mixt_value(m[j],S[j],P[j],X[:,i])
            temp.append(t)
        temp=P_cl*temp
        q1=np.max(temp)
        q2=np.argmax(temp)
        y.append(q2+1)
    return y

def cls_error(m_ini,S_ini,w_ini):
    [m_hat,S_hat,w_hat,P_hat]=EM_pdf_est(np.concatenate([X1,X2],1),np.concatenate([np.ones([500]),2*np.ones([500])]),m_ini,S_ini,w_ini)
    # 2. Use function mixture_Bayes to classify the data vectors of Z and function compute_error to % obtain the classification error.
    S=[]
    for j in range(2):
        le=len(S_hat[j])
        te=np.zeros((2,2,le))
        for i in range(le):
            te[:,:,i]=S_hat[j][i]*np.eye(2)
        S.append(te)

    y_est=mixture_Bayes(m_hat,S,w_hat,P_hat,Z)
    y_est=np.array(y_est)

    real_label=np.concatenate([np.ones([500]),2*np.ones([500])])
    classification_error=np.abs(np.sum(y_est-real_label))/1000
    print(classification_error)

Generate the subset X1 of X, which contains the data points from the first class

In [17]:
m1=np.array([[1.25,1.25], [2.75,2.75], [2,6]]).T
S1=np.transpose(np.array([0.1*I,0.2*I,0.3*I]),[2,1,0])
P1=[0.4, 0.4,0.2]
N1=500
X1,y1=mixt_model(m1,S1,P1,N1)

The subset X2 of X, with the points from the second class is generated similarly


In [18]:
m2=np.array([[1.25, 2.75], [2.75 ,1.25], [4 ,6]]).T
S2=np.transpose(np.array([0.1*I,0.2*I,0.3*I]),[2,1,0])
P2=[0.2, 0.3,0.5]
N2=500
X2,y2=mixt_model(m2,S2,P2,N2)

Dataset Z is generated in two steps,i.e., 500 points are first generated from 

In [19]:
mZ1=np.array([[1.25,1.25], [2.75,2.75], [2,6]]).T
SZ1=np.transpose(np.array([0.1*I,0.2*I,0.3*I]),[2,1,0])
PZ1=[0.4, 0.4,0.2]
NZ1=500
Z1,yZ1=mixt_model(mZ1,SZ1,PZ1,NZ1)

mZ2=np.array([[1.25, 2.75], [2.75 ,1.25], [4 ,6]]).T
SZ2=np.transpose(np.array([0.1*I,0.2*I,0.3*I]),[2,1,0])
PZ2=[0.2, 0.3,0.5]
NZ2=500
Z2,yZ2=mixt_model(mZ2,SZ2,PZ2,NZ2)

Z=np.concatenate([Z1,Z2],1)

1. Estimate the Gaussian mixture model of each class

In [20]:
#part 1

m1_ini =np.array([[5, 5],[5.5,5.5],[5, 5]]).T
S1_ini=np.array([.2,0.4,0.3])
w1_ini=np.array([.2, .4, .4])

m2_ini=np.array([[2, 2],[1.98,1.98],[2.4,2.4]]).T
S2_ini=np.array([.06,0.05,0.4])
w2_ini=np.array([.8, .1, .1])

m_ini=[m1_ini,m2_ini]
S_ini=[S1_ini,S2_ini]
w_ini=[w1_ini,w2_ini]


cls_error(m_ini,S_ini,w_ini)

0.128


In [21]:
#part 2

m1_ini =np.array([[1.6, 1.4],[1.4,1.6],[1.3, 1.5]]).T
S1_ini=np.array([.2,0.4,0.3])
w1_ini=np.array([.2, .4, .4])

m2_ini=np.array([[1.5, 1.7],[1.7, 1.5],[1.6, 1.6]]).T
S2_ini=np.array([.6,0.05,0.02])
w2_ini=np.array([.1, .8, .1])

m_ini=[m1_ini,m2_ini]
S_ini=[S1_ini,S2_ini]
w_ini=[w1_ini,w2_ini]


cls_error(m_ini,S_ini,w_ini)

0.127


In [22]:
#part 3

m1_ini =np.array([[0, 2],[5, 2],[5, 5],[3,4] ]).T
S1_ini=np.array([.15,0.27,0.4, .2])
w1_ini=np.array([1/4, 1/4, 1/4, 1/4 ])

m2_ini=np.array([[1, 2],[3.2, 1.5],[1, 4],[4,2] ]).T
S2_ini=np.array([.15,0.08,0.27,.05])
w2_ini=np.array([1/4, 1/4, 1/4, 1/4 ])

m_ini=[m1_ini,m2_ini]
S_ini=[S1_ini,S2_ini]
w_ini=[w1_ini,w2_ini]


cls_error(m_ini,S_ini,w_ini)

0.077


In [23]:
#part 4

m1_ini =np.array([[0, 2],[5, 2]]).T
S1_ini=np.array([.15,.27])
w1_ini=np.array([1/2,1/2])

m2_ini=np.array([[1, 2]]).T
S2_ini=np.array([.15])
w2_ini=np.array([1])

m_ini=[m1_ini,m2_ini]
S_ini=[S1_ini,S2_ini]
w_ini=[w1_ini,w2_ini]


cls_error(m_ini,S_ini,w_ini)

0.011


In [24]:
#part 5

m1_ini =np.array([[2, 2]]).T
S1_ini=np.array([0.4])
w1_ini=np.array([1])

m2_ini=np.array([[1, 2]]).T
S2_ini=np.array([.15])
w2_ini=np.array([1])

m_ini=[m1_ini,m2_ini]
S_ini=[S1_ini,S2_ini]
w_ini=[w1_ini,w2_ini]


cls_error(m_ini,S_ini,w_ini)

0.091
